In [7]:
%matplotlib inline
from psycopg2 import connect
import psycopg2.sql as pg
import configparser
from datetime import datetime, timedelta, date
from itertools import cycle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#import matplotlib.cm as cm
#from matplotlib.ticker import FuncFormatter
import pandas as pd
import pandas.io.sql as pandasql
import seaborn as sns
from IPython.display import HTML
def print_table(sql, con):
    return HTML(pandasql.read_sql(sql, con).to_html(index=False))

In [8]:
# setting up pgsql connection
CONFIG = configparser.ConfigParser()
CONFIG.read(r'/home/jchew/local/db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [9]:
# Select bt speeds
sql = '''SELECT * FROM jchew.bt_top10_2days_spd'''
print_table(sql, con)

analysis_id,bt_id,street_name,direction,from_intersection,to_intersection,length,median_tt,speed
1453262,6,Dundas,EB,Dufferin,Bathurst,2088,386.00,19.473575
1453284,7,Dundas,EB,Bathurst,Spadina,675,112.00,21.696429
1453305,8,Dundas,EB,Spadina,University,806,165.50,17.532326
1453367,11,Dundas,EB,Jarvis,Parliament,750,119.50,22.594142
1453395,12,Dundas,WB,Parliament,Jarvis,754,135.50,20.032472
1453445,15,Dundas,WB,University,Spadina,806,168.00,17.271429
1453464,16,Dundas,WB,Spadina,Bathurst,675,98.00,24.795918
1453483,17,Dundas,WB,Bathurst,Dufferin,2083,366.00,20.488525
1454196,53,King,EB,Spadina,University,865,199.00,15.648241
1454209,54,King,EB,University,Yonge,573,144.00,14.325000


In [10]:
# Finding ratio for bt speeds
sql = '''
WITH X AS (
SELECT a.analysis_id AS analysis_id_1, a.bt_id AS bt_id_1, 
	b.analysis_id AS analysis_id_2, b.bt_id AS bt_id_2, a.street_name, 
	a.direction AS eb_nb, b.direction AS wb_sb, 
	a.from_intersection AS intersection_1, a.to_intersection AS intersection_2,
	a.speed AS eb_nb_spd, b.speed AS wb_sb_spd
	FROM jchew.bt_top10_2days_spd a
JOIN jchew.bt_top10_2days_spd b
ON a.street_name = b .street_name AND a.from_intersection = b.to_intersection 
AND a.to_intersection=b.from_intersection 
WHERE a.direction IN ('EB', 'NB')
)
SELECT *,
CASE WHEN X.eb_nb_spd > X.wb_sb_spd THEN (X.eb_nb_spd * 1.0) / (X.eb_nb_spd + X.wb_sb_spd)
WHEN X.eb_nb_spd < X.wb_sb_spd THEN (X.wb_sb_spd * 1.0) / (X.eb_nb_spd + X.wb_sb_spd)
END AS "EB/WB or NB/SB Ratio",
 CASE
WHEN X.eb_nb = 'EB' THEN CASE
	WHEN X.eb_nb_spd < X.wb_sb_spd THEN 'WB'
    WHEN X.eb_nb_spd > X.wb_sb_spd THEN 'EB'
    END
WHEN X.eb_nb = 'NB' THEN CASE
    WHEN X.eb_nb_spd < X.wb_sb_spd THEN 'SB'
    WHEN X.eb_nb_spd > X.wb_sb_spd THEN 'NB'
    END
        END AS "Bias Towards"
FROM X
ORDER BY analysis_id_1
'''
print_table(sql, con)

analysis_id_1,bt_id_1,analysis_id_2,bt_id_2,street_name,eb_nb,wb_sb,intersection_1,intersection_2,eb_nb_spd,wb_sb_spd,EB/WB or NB/SB Ratio,Bias Towards
1453262,6,1453483,17,Dundas,EB,WB,Dufferin,Bathurst,19.473575,20.488525,0.512699,WB
1453284,7,1453464,16,Dundas,EB,WB,Bathurst,Spadina,21.696429,24.795918,0.533333,WB
1453305,8,1453445,15,Dundas,EB,WB,Spadina,University,17.532326,17.271429,0.503748,EB
1453367,11,1453395,12,Dundas,EB,WB,Jarvis,Parliament,22.594142,20.032472,0.530048,EB
1454196,53,1454366,62,King,EB,WB,Spadina,University,15.648241,11.081851,0.585417,EB
1454209,54,1454352,61,King,EB,WB,University,Yonge,14.325000,10.471066,0.577713,EB
1454523,70,1454683,78,Front,EB,WB,Bathurst,Spadina,23.448128,29.627027,0.558209,WB
1454549,71,1454670,77,Front,EB,WB,Spadina,University,14.980165,13.550562,0.525054,EB
1455385,117,1455256,110,University,NB,SB,Queen,Dundas,27.034483,22.142466,0.549739,NB


In [11]:
# Select here observations (no king)
sql = '''SELECT * FROM jchew.here_top10_2days_no_king_spd
ORDER BY street_name'''
print_table(sql, con)

analysis_id,street_name,direction,from_intersection,to_intersection,speed
1453367,Dundas,EB,Jarvis,Parliament,28.137216
1453483,Dundas,WB,Bathurst,Dufferin,27.739099
1453284,Dundas,EB,Bathurst,Spadina,28.338286
1453305,Dundas,EB,Spadina,University,23.897257
1453262,Dundas,EB,Dufferin,Bathurst,27.371887
1453395,Dundas,WB,Parliament,Jarvis,26.187566
1453445,Dundas,WB,University,Spadina,23.180233
1453464,Dundas,WB,Spadina,Bathurst,28.576609
1454549,Front,EB,Spadina,University,22.489108
1454670,Front,WB,University,Spadina,21.693694


In [12]:
sql = '''
WITH X AS (
SELECT a.analysis_id AS analysis_id_1,
	b.analysis_id AS analysis_id_2, a.street_name, 
	a.direction AS eb_nb, b.direction AS wb_sb, 
	a.from_intersection AS intersection_1, a.to_intersection AS intersection_2,
	a.speed AS eb_nb_spd, b.speed AS wb_sb_spd
	FROM jchew.here_top10_2days_no_king_spd a
JOIN jchew.here_top10_2days_no_king_spd b
ON a.street_name = b .street_name AND a.from_intersection = b.to_intersection 
AND a.to_intersection=b.from_intersection 
WHERE a.direction IN ('EB', 'NB')
)
SELECT *,
CASE WHEN X.eb_nb_spd > X.wb_sb_spd THEN (X.eb_nb_spd * 1.0) / (X.eb_nb_spd + X.wb_sb_spd)
WHEN X.eb_nb_spd < X.wb_sb_spd THEN (X.wb_sb_spd * 1.0) / (X.eb_nb_spd + X.wb_sb_spd)
END AS "EB/WB or NB/SB Ratio",
 CASE
WHEN X.eb_nb = 'EB' THEN CASE
	WHEN X.eb_nb_spd < X.wb_sb_spd THEN 'WB'
    WHEN X.eb_nb_spd > X.wb_sb_spd THEN 'EB'
    END
WHEN X.eb_nb = 'NB' THEN CASE
    WHEN X.eb_nb_spd < X.wb_sb_spd THEN 'SB'
    WHEN X.eb_nb_spd > X.wb_sb_spd THEN 'NB'
    END
        END AS "Bias Towards"
FROM X
ORDER BY analysis_id_1
'''
print_table(sql, con)

analysis_id_1,analysis_id_2,street_name,eb_nb,wb_sb,intersection_1,intersection_2,eb_nb_spd,wb_sb_spd,EB/WB or NB/SB Ratio,Bias Towards
1453262,1453483,Dundas,EB,WB,Dufferin,Bathurst,27.371887,27.739099,0.503332,WB
1453284,1453464,Dundas,EB,WB,Bathurst,Spadina,28.338286,28.576609,0.502094,WB
1453305,1453445,Dundas,EB,WB,Spadina,University,23.897257,23.180233,0.507615,EB
1453367,1453395,Dundas,EB,WB,Jarvis,Parliament,28.137216,26.187566,0.517944,EB
1454196,1454366,King,EB,WB,Spadina,University,NaN,NaN,NaN,None
1454209,1454352,King,EB,WB,University,Yonge,NaN,NaN,NaN,None
1454523,1454683,Front,EB,WB,Bathurst,Spadina,29.331858,28.509690,0.507107,EB
1454549,1454670,Front,EB,WB,Spadina,University,22.489108,21.693694,0.509001,EB
1455385,1455256,University,NB,SB,Queen,Dundas,35.342676,31.951506,0.525197,NB
1455400,1455243,University,NB,SB,Dundas,College,34.922414,29.254248,0.544161,NB


In [14]:
# To compare ratios for both bt and here (NO KING)
sql = '''
SELECT b.analysis_id_1, b.analysis_id_2, b.street_name, 
b.eb_nb, b.wb_sb, b.intersection_1, b.intersection_2, 
a.eb_nb_spd AS bt_eb_nb_spd, a.wb_sb_spd AS bt_wb_sb_spd,
b.eb_nb_spd AS here_eb_nb_spd, b.wb_sb_spd AS here_wb_sb_spd,
a."EB/WB or NB/SB Ratio" AS bt_ratio,
a."Bias Towards" AS bt_bias,
b."EB/WB or NB/SB Ratio" AS here_ratio,
b."Bias Towards" AS here_bias
FROM jchew.ratio_bt_top10_2days_spd a
RIGHT JOIN jchew.ratio_here_top10_2days_no_king_spd b
USING (analysis_id_1)
'''
print_table(sql, con)

analysis_id_1,analysis_id_2,street_name,eb_nb,wb_sb,intersection_1,intersection_2,bt_eb_nb_spd,bt_wb_sb_spd,here_eb_nb_spd,here_wb_sb_spd,bt_ratio,bt_bias,here_ratio,here_bias
1453262,1453483,Dundas,EB,WB,Dufferin,Bathurst,19.473575,20.488525,27.371887,27.739099,0.512699,WB,0.503332,WB
1453284,1453464,Dundas,EB,WB,Bathurst,Spadina,21.696429,24.795918,28.338286,28.576609,0.533333,WB,0.502094,WB
1453305,1453445,Dundas,EB,WB,Spadina,University,17.532326,17.271429,23.897257,23.180233,0.503748,EB,0.507615,EB
1453367,1453395,Dundas,EB,WB,Jarvis,Parliament,22.594142,20.032472,28.137216,26.187566,0.530048,EB,0.517944,EB
1454196,1454366,King,EB,WB,Spadina,University,15.648241,11.081851,NaN,NaN,0.585417,EB,NaN,None
1454209,1454352,King,EB,WB,University,Yonge,14.325000,10.471066,NaN,NaN,0.577713,EB,NaN,None
1454523,1454683,Front,EB,WB,Bathurst,Spadina,23.448128,29.627027,29.331858,28.509690,0.558209,WB,0.507107,EB
1454549,1454670,Front,EB,WB,Spadina,University,14.980165,13.550562,22.489108,21.693694,0.525054,EB,0.509001,EB
1455385,1455256,University,NB,SB,Queen,Dundas,27.034483,22.142466,35.342676,31.951506,0.549739,NB,0.525197,NB
1455400,1455243,University,NB,SB,Dundas,College,NaN,NaN,34.922414,29.254248,NaN,None,0.544161,NB
